In [430]:
import mailparser
import re
import pandas as pd
import io
import glob
import numpy as np

In [444]:

class EmlFileParser(object):
    def __init__(self, file):
        super().__init__()
        self.file = file   

    def parse_metropolitaine(self):
        df = self.parse()
        df = df.filter(regex='SITREP|^J\s|^L\s|^N\s')
        df.iloc[:,0] =df.columns[0]
        df = df.rename(columns={df.columns[0]: "cross_sitrep"})
        df['cross_sitrep'] = df.cross_sitrep.str.replace('=C9', "E").str.extract("(?:MRCC|CROSS|JRCC) ((?:LA*\s)*[A-Z]+\/\d+/(?:N°)*\d+)")
        df = self.colname_homogeneous(df)
        return df
  
    def parse_tahiti(self):
        df = self.parse()
        df = df.filter(regex='BT|JRCC|^J-|^L-|^N-')
        df.iloc[:,0] =df.columns[0]
        df = df.rename(columns={df.columns[0]: "cross_sitrep"})
        df['cross_sitrep'] = "TAHITI/"+df['BT'].str.extract('(\d+/\d+)')
        df = df.drop(['BT', 'JRCC TAHITI'], axis=1)
        df = self.colname_homogeneous(df)
        return df
    
    def parse_reunion(self):
        df = self.parse()
        df = df.filter(regex='FM CROSS REUNION|^J-|^L-|^N-')
        df.iloc[:,0] =df.columns[0]
        df['cross_sitrep'] = df['FM CROSS REUNION'].str.extract('(?:CROSS) (R=C9UNION/\d+/\d+)').str.replace('=C9', 'E')
        df = df.drop('FM CROSS REUNION', axis=1)
        df = self.colname_homogeneous(df)
        return df
        
    def colname_homogeneous(self, df):
        di={'J - INITIAL ACTIONS TAKEN': 'J - PREMIERES MESURES PRISES',
           'J - PREMIÈRES MESURES PRISES':'J - PREMIERES MESURES PRISES',
           'J – PREMIERES MESURES PRISES':'J - PREMIERES MESURES PRISES',
           'J-    PREMIERES MESURES PRISES':'J - PREMIERES MESURES PRISES',
           'L - CHRONOLOGIE':'L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION',
           'L - COORDINATING INSTRUCTIONS':'L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION',
           'L-    INSTRUCTIONS POUR LA COORDINATION':'L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION',
           'L – CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION':'L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION',
           'N - ADDITIONAL INFORMATION':'N - RENSEIGNEMENTS COMPLEMENTAIRES',
           'N - RENSEIGNEMENTS COMPLÉMENTAIRES':'N - RENSEIGNEMENTS COMPLEMENTAIRES',
           'N - RENSEIGNEMENTS COMPLMENTAIRES':'N - RENSEIGNEMENTS COMPLEMENTAIRES',
           'N – RENSEIGNEMENTS COMPLEMENTAIRES':'N - RENSEIGNEMENTS COMPLEMENTAIRES',
           'N-    RENSEIGNEMENTS COMPLEMENTAIRES':'N - RENSEIGNEMENTS COMPLEMENTAIRES'}
        return df.rename(columns=lambda x: x.strip()).rename(columns=di)
      
    def parse(self):
        data = self.text_to_df()
        data.columns =['lines'] #Name the dataframe column
        data['A'], data['B'] = data['lines'].str.split('\n',1).str #Split lines 
        data = data.drop_duplicates('A', keep='first') #Drop duplicated lines
        data = data.transpose()# Transpose the Data frame 
        data.columns = data.iloc[1] #Name colums with info in row A
        data = data.reindex(data.index.drop(['A','lines'])).reset_index() 
        return data
    
    def eml_to_text(self):
        mail = mailparser.parse_from_file(self.file)
        return mail.body

    def text_to_df(self):
        a =  self.eml_to_text().split('\n\n')
        return pd.DataFrame(a)



class EmlFolderParser(object):
    def __init__(self, folder, output):
        super().__init__()
        self.folder = folder
        self.output = output
    
    def parse_folder(self):
        text_df = pd.DataFrame()
        for filename in glob.glob(self.folder+'/*.eml'):
            if ('TAHITI' in filename)== True:            
                out = EmlFileParser(filename).parse_tahiti()
            elif ('UNION' in filename)== True:
                out = EmlFileParser(filename).parse_reunion()
            else:
                out = EmlFileParser(filename).parse_metropolitaine()
            text_df = text_df.append(out, ignore_index=True)
        text_df.to_csv(self.output)
    

In [450]:
#EmlFolderParser('./liste_sitrep', './liste_sitrep/test_all3.csv').parse_folder()

In [313]:
def parse_sitrep(input_path, output_path):
    text_df = pd.DataFrame() 
    for filename in glob.glob(input_path+'/*.eml'):
        mail = mailparser.parse_from_file(filename)
        text = mail.body
        a = text.split('\n\n')
        df = pd.DataFrame(a)
        df.columns =['test']
        df['A'], df['B'] = df['test'].str.split('\n',1).str 
        df = df.drop_duplicates('A', keep='first')
        df1 = df.transpose()
        df1.columns = df1.iloc[1]
        df2 = df1.reindex(df1.index.drop('A')).reset_index()
        df3 = df2.filter(regex='SITREP|^J\s|^L\s|^N\s').drop(0)
        df3.iloc[:,0] =df3.columns[0]
        df3 = df3.rename(columns={ df3.columns[0]: "cross_sitrep"})
        df3['cross_sitrep'] = df3.cross_sitrep.str.extract("(?:MRCC|CROSS|JRCC) ([A-Z]+\/\d+/(?:N°)*\d+)")#ajouter un truc pour La garde 
        text_df = text_df.append(df3, ignore_index=True)
    text_df.to_csv(output_path)


    

In [357]:
parse_sitrep('./liste_sitrep', './liste_sitrep/test.csv')

/Users/elsa.luneville/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [458]:
class EmlFileParserTest(object):
    def __init__(self, file):
        super().__init__()
        self.file = file   

    def parse_metropolitaine(self):
        df = self.parse()
        df = df.filter(regex='SITREP|^J\s|^L\s|^N\s')
        df.iloc[:,0] =df.columns[0]
        df = df.rename(columns={df.columns[0]: "cross_sitrep"})
        df['cross_sitrep'] = df.cross_sitrep.str.replace('=C9', "E").str.extract("(?:MRCC|CROSS|JRCC) ((?:LA*\s)*[A-Z]+\/\d+/(?:N°)*\d+)")
        df = self.colname_homogeneous(df)
        return df
  
    def parse_tahiti(self):
        df = self.parse()
        df = df.filter(regex='|BT|JRCC|^J-|^L-|^N-')
      #  df.iloc[:,0] =df.columns[0]
       # df = df.rename(columns={df.columns[0]: "cross_sitrep"})
       # df['cross_sitrep'] = "TAHITI/"+df['BT'].str.extract('(\d+/\d+)')
       # df = df.drop(['BT', 'JRCC TAHITI'], axis=1)
        #df = self.colname_homogeneous(df)
        return df
    
    def parse_reunion(self):
        df = self.parse()
        df = df.filter(regex='FM CROSS REUNION|^J-|^L-|^N-')
        df.iloc[:,0] =df.columns[0]
        df['cross_sitrep'] = df['FM CROSS REUNION'].str.extract('(?:CROSS) (R=C9UNION/\d+/\d+)').str.replace('=C9', 'E')
        df = df.drop('FM CROSS REUNION', axis=1)
        df = self.colname_homogeneous(df)
        return df
        
    def colname_homogeneous(self, df):
        di={'J - INITIAL ACTIONS TAKEN': 'J - PREMIERES MESURES PRISES',
           'J - PREMIÈRES MESURES PRISES':'J - PREMIERES MESURES PRISES',
           'J – PREMIERES MESURES PRISES':'J - PREMIERES MESURES PRISES',
           'J-    PREMIERES MESURES PRISES':'J - PREMIERES MESURES PRISES',
           'L - CHRONOLOGIE':'L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION',
           'L - COORDINATING INSTRUCTIONS':'L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION',
           'L-    INSTRUCTIONS POUR LA COORDINATION':'L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION',
           'L – CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION':'L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION',
           'N - ADDITIONAL INFORMATION':'N - RENSEIGNEMENTS COMPLEMENTAIRES',
           'N - RENSEIGNEMENTS COMPLÉMENTAIRES':'N - RENSEIGNEMENTS COMPLEMENTAIRES',
           'N - RENSEIGNEMENTS COMPLMENTAIRES':'N - RENSEIGNEMENTS COMPLEMENTAIRES',
           'N – RENSEIGNEMENTS COMPLEMENTAIRES':'N - RENSEIGNEMENTS COMPLEMENTAIRES',
           'N-    RENSEIGNEMENTS COMPLEMENTAIRES':'N - RENSEIGNEMENTS COMPLEMENTAIRES'}
        return df.rename(columns=lambda x: x.strip()).rename(columns=di)
      
    def parse(self):
        data = self.text_to_df()
        data.columns =['lines'] #Name the dataframe column
        data['A'], data['B'] = data['lines'].str.split('\n',1).str #Split lines 
        data = data.drop_duplicates('A', keep='first') #Drop duplicated lines
        data = data.transpose()# Transpose the Data frame 
        data.columns = data.iloc[1] #Name colums with info in row A
        data = data.reindex(data.index.drop(['A','lines'])).reset_index() 
        return data
    
    def eml_to_text(self):
        mail = mailparser.parse_from_file(self.file)
        return mail.body

    def text_to_df(self):
        a =  self.eml_to_text().split('\n\n')
        return pd.DataFrame(a)



class EmlFolderParser(object):
    def __init__(self, folder, output):
        super().__init__()
        self.folder = folder
        self.output = output
    
    def parse_folder(self):
        text_df = pd.DataFrame()
        for filename in glob.glob(self.folder+'/*.eml'):
            if ('TAHITI' in filename)== True:            
                out = EmlFileParser(filename).parse_tahiti()
            elif ('UNION' in filename)== True:
                out = EmlFileParser(filename).parse_reunion()
            else:
                out = EmlFileParser(filename).parse_metropolitaine()
            text_df = text_df.append(out, ignore_index=True)
        text_df.to_csv(self.output)

In [459]:
EmlFileParserTest('./liste_sitrep/JRCC TAHITI - SITREP SAR 406-2017- UN ET FINAL.eml').parse_tahiti()

A,index,FM JRCC TAHITI,,A- NATURE DE L'EVENEMENT ET IDENTITE DU MOBILE IMPLIQUE,B- POSITION,C- SITUATION INITIALE,D- NOMBRE DE PERSONNES IMPLIQUEES,E- ASSISTANCE DEMANDEE,F- CENTRE(S) DE COORDINATION,G- DESCRIPTION DU MOBILE,J- PREMIERES MESURES PRISES,K- ZONES DE RECHERCHE,L- INSTRUCTIONS POUR LA COORDINATION,M- PLANS FUTURS,N- RENSEIGNEMENTS COMPLEMENTAIRES,--,JRCC TAHITI,BP 9420 – 98715 Papeete CMP
0,B,TO HAUSSAIRE PF\nDAM SM1\nSG MER\nCOFGC\nCMVOA...,BT\nNON PROTEGE\nMCA SECMAR\nOBJ/SITREP SAR 40...,CHAVIREMENT – PIROGUE TYPE V6,17°31S / 149°34W – PAPEETE - ILE DE TAHITI,LE 04/12/2017 A 17H00 – APPEL TELEPHONIQUE D’U...,6,RECHERCHE ET SAUVETAGE,JRCC TAHITI,PIROGUE DE COURSE TYPE V6\nH- CONDITI...,17H05 – DIFFUSION VHF 16 MESSAGE PAN-PAN...,NEANT,17H19 – 4 PERSONNES ONT ATTEINT LE RECIF...,NEANT,18H20 – OPERATION CLOSE\n 1 PIROGUE...,[cid:part1.39C5180B.CE0C4293@mail.pf],Centre de coordination de sauvetage aéro marit...,Tél : Urgence 24/24 : 16 – International : (+6...


In [161]:
#Handle cross reunion
#Add message error
#Split parsing function
#make test

C:\Users\elsal\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


A,cross_sitrep,L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION,N - RENSEIGNEMENTS COMPLEMENTAIRES
0,NaN,LE 26/12/2017\n03:25 – SUITE NOMBREUX APPELS P...,09:47 - OPÉRATION CLOSE\n01 NAVIRE REMORQUÉ\nBT
